<a href="https://colab.research.google.com/github/sumanthd17/aspect-based-sentiment/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 7.4MB/s 
     |████████████████████████████████| 1.1MB 22.5MB/s 
     |████████████████████████████████| 3.0MB 20.0MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=8c1c8080471b1a72a6a923ecb30ec5bac45053cd13c90dce9b858fdd16141fe2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
import argparse

import pandas as pd
from tqdm import tqdm

import torch
import transformers as optimus

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def load_train_data(input_dir):
    df = pd.read_csv(input_dir + "train-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [48]:
def hyper_params():
    BATCH_SIZE = 32
    MAX_SEQ_LENGTH = 512
    return BATCH_SIZE, MAX_SEQ_LENGTH

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_class, tokenizer_class, pretrained_weights = (
    optimus.BertForSequenceClassification,
    optimus.BertTokenizer,
    "bert-base-uncased",
)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
!python create_data.py

2977
(1430, ['LOCATION1', 'is', 'transforming', 'and', 'the', 'prices', 'will', 'go', 'up', 'and', 'up'], 'LOCATION1', 'price', 'Negative')
45025


In [32]:
train_data = load_train_data('QA_pairs/')

batch_size, max_seq_len = hyper_params()
num_training_steps = int(len(train_data) / batch_size)
print(len(train_data))

45025


In [7]:
tokenizer.encode('hi! my name is sumanth', add_special_tokens=True)

[101, 7632, 999, 2026, 2171, 2003, 7680, 4630, 2232, 102]

In [27]:
tokenizer.tokenize('what do you think about the location1 and the price of location2')

['what',
 'do',
 'you',
 'think',
 'about',
 'the',
 'location',
 '##1',
 'and',
 'the',
 'price',
 'of',
 'location',
 '##2']

In [19]:
train_sample = train_data[:10000]

In [71]:
tokenized = pd.DataFrame()

In [72]:
tokenized['input_ids'] = train_sample['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [73]:
tokenized['input_mask'] = ''
for i, row in tokenized.iterrows():
  tokenized.iloc[i]['input_mask'] = [1]*len(tokenized.iloc[i]['input_ids'])

In [74]:
tokenized['input_mask'] = tokenized['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [75]:
tokenized['input_ids'] = tokenized['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

In [76]:
sent2idx = {
    'None': 0,
    'Positive': 1,
    'Negative': 2
}

In [77]:
tokenized['label'] = ''
tokenized['label'] = train_sample['sentiment'].apply(lambda x: sent2idx[x])

In [78]:
train_data['sentiment'].value_counts()

None        41629
Positive     2475
Negative      921
Name: sentiment, dtype: int64

In [84]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [79]:
input_ids = torch.tensor([r for r in tokenized['input_ids']], dtype=torch.long)
input_masks = torch.tensor([r for r in tokenized['input_mask']], dtype=torch.long)
label_ids = torch.tensor([r for r in tokenized['label']], dtype=torch.long)

In [82]:
train_dataset = TensorDataset(input_ids, input_masks, label_ids)

In [86]:
train_sampler = RandomSampler(train_dataset)

In [87]:
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)